# Correlation Analysis

In [145]:
import pandas as pd

In [146]:
# Load the ducument
df=pd.read_csv('FOMC_classification_results.csv')

In [147]:
#Cheak the dataframe
df.head()

,date,text,dovish_count,hawkish_count,classification,classification_numeric,hawkish_similarity,dovish_similarity,classification_s,classification_s_numeric
0,2011-01-26,"The Federal Reserve, the central bank of the U...",7,13,hawkish,1,0.449360,0.650187,neutral,0
1,2011-03-15,"The Federal Reserve, the central bank of the U...",6,17,hawkish,1,0.424052,0.622608,neutral,0
2,2011-04-27,"The Federal Reserve, the central bank of the U...",6,16,hawkish,1,0.424086,0.621737,neutral,0
3,2011-06-22,"The Federal Reserve, the central bank of the U...",5,12,hawkish,1,0.503104,0.660841,hawkish,1
4,2011-08-09,"The Federal Reserve, the central bank of the U...",6,12,hawkish,1,0.540741,0.665799,hawkish,1


In [148]:
# Load the bond market data from Excel file
xls = pd.ExcelFile('./FOMC_Data_2011_2024.xlsx')
gt10_df = pd.read_excel(xls, 'GT10')
gt2_df = pd.read_excel(xls, 'GT2')
spread_df = pd.read_excel(xls, '2s10s_Spread')
gold_df = pd.read_excel(xls, 'Gold_Prices')
vix_df = pd.read_excel(xls, 'VIX')
sp500_df = pd.read_excel(xls, 'SP500')

In [149]:
sp500_df.tail()

,Date,PX_LAST,Change,% Change,10day
3206,2012-01-06,1277.81,NaN,NaN,0.017803
3207,2012-01-05,1281.06,NaN,NaN,0.017185
3208,2012-01-04,1277.30,NaN,NaN,0.016916
3209,2012-01-03,1277.06,NaN,NaN,0.022316
3210,2011-12-30,1257.61,NaN,NaN,0.000000


In [150]:
import pandas as pd
import numpy as np

def extract_data(target_df, source_df):
    # Ensure date columns are datetime type
    target_df.iloc[:, 0] = pd.to_datetime(target_df.iloc[:, 0], errors='coerce')
    source_df.iloc[:, 0] = pd.to_datetime(source_df.iloc[:, 0], errors='coerce')
    
    # Create a dictionary with dates as keys and corresponding data as values
    data_dict = dict(zip(source_df.iloc[:, 0], source_df.iloc[:, -1]))
    
    # Use target DataFrame's dates to look up data, replace with 0 if not found
    extracted_data = [data_dict.get(date, 0) for date in target_df.iloc[:, 0]]
    
    return extracted_data

# Assume df is our benchmark DataFrame containing the dates we want
# Ensure df only contains the date column and the last data column
df = df.iloc[:, [0, -1]]
df.columns = ['Date', 'Benchmark']

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Extract data for the other 6 DataFrames
other_dfs = [gt10_df, gt2_df, spread_df, gold_df, vix_df, sp500_df]
df_names = ['GT10', 'GT2', 'Spread', 'Gold', 'VIX', 'SP500']

for source_df, name in zip(other_dfs, df_names):
    df[name] = extract_data(df, source_df)

# Filter out rows with more than 3 zeros
df = df[df.iloc[:, 1:].eq(0).sum(axis=1) <= 3]

# Format the date to remove the time component
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# Print results
print(f"Final data shape: {df.shape}")
print(df.tail(100))
print("\nData types:")
print(df.dtypes)

Final data shape: (820, 8)
           Date  Benchmark      GT10       GT2    Spread      Gold       VIX  \
754  2023-11-09          0 -0.046414 -0.009466  0.667528 -0.006644 -0.181648   
755  2023-11-13         -1 -0.046836 -0.014900  0.478945  0.002782 -0.143765   
756  2023-11-14          1 -0.047070 -0.017517  0.404035  0.008700 -0.129458   
757  2023-11-16          1 -0.041558 -0.019255  0.260006  0.019069 -0.108161   
758  2023-11-16         -1 -0.041558 -0.019255  0.260006  0.019069 -0.108161   
..          ...        ...       ...       ...       ...       ...       ...   
853  2024-09-10         -1 -0.032873 -0.057089 -7.230718  0.025917 -0.058956   
854  2024-09-10         -1 -0.032873 -0.057089 -7.230718  0.025917 -0.058956   
855  2024-09-18         -1  0.019164 -0.014188  3.504585  0.039726 -0.119447   
856  2024-09-18         -1  0.019164 -0.014188  3.504585  0.039726 -0.119447   
857  2024-09-20          1  0.026659 -0.002020  1.810909  0.040923 -0.060538   

        SP50

In [157]:
# Print results
print(f"Final data shape: {df.shape}")
print(df.head())
print("\nData types:")
print(df.dtypes)

Final data shape: (820, 8)
          Date  Benchmark      GT10       GT2    Spread      Gold       VIX  \
8   2012-01-06          1 -0.008991 -0.073354  0.000762  0.029253 -0.084890   
9   2012-01-06          1 -0.008991 -0.073354  0.000762  0.029253 -0.084890   
11  2012-01-13          1  0.006962 -0.075739  0.019267  0.035396 -0.094618   
12  2012-01-16          0  0.009082 -0.073031  0.021194  0.036841  0.000000   
13  2012-02-10          1  0.045770  0.244511  0.018007  0.001795  0.026201   

       SP500  
8   0.017803  
9   0.017803  
11  0.023356  
12  0.000000  
13  0.017023  

Data types:
Date          object
Benchmark      int64
GT10         float64
GT2          float64
Spread       float64
Gold         float64
VIX          float64
SP500        float64
dtype: object


In [158]:
#Save it to a csv file for convenience 
df.to_csv('Ready for analysis.csv', index=False)

In [154]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
data = df

In [156]:
# Select all columns except 'Date' for correlation analysis
columns_for_correlation = [col for col in data.columns if col != 'Date']
correlation_data = data[columns_for_correlation]

# Calculate the correlation matrix
correlation_matrix = correlation_data.corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.savefig('correlation_heatmap.png')
plt.close()

# Create a pairplot
sns.pairplot(correlation_data, diag_kind='kde')
plt.tight_layout()
plt.savefig('pairplot.png')
plt.close()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)


Correlation Matrix:
           Benchmark      GT10       GT2    Spread      Gold       VIX  \
Benchmark   1.000000  0.073902  0.046069  0.046390 -0.018340 -0.052347   
GT10        0.073902  1.000000  0.681866 -0.045822 -0.367307 -0.335362   
GT2         0.046069  0.681866  1.000000 -0.044409 -0.371492 -0.300495   
Spread      0.046390 -0.045822 -0.044409  1.000000 -0.006796  0.021830   
Gold       -0.018340 -0.367307 -0.371492 -0.006796  1.000000  0.029885   
VIX        -0.052347 -0.335362 -0.300495  0.021830  0.029885  1.000000   
SP500       0.039623  0.203457  0.225608  0.024386  0.053180 -0.775559   

              SP500  
Benchmark  0.039623  
GT10       0.203457  
GT2        0.225608  
Spread     0.024386  
Gold       0.053180  
VIX       -0.775559  
SP500      1.000000  

Strong Correlations (|correlation| > 0.5):
GT2 and GT10: 0.68
SP500 and VIX: -0.78
